### Задание №1
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [1]:
import pandas as pd

In [4]:
ratings = pd.read_csv(r'C:\Users\PC-Home\Desktop\Data_analyst\HW-8_pandas_joins\ml-latest-small\ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [17]:
ratings_grouped = ratings.groupby('userId').count().reset_index()
ratings_filtered = ratings_grouped[ratings_grouped['rating']>=100]

# Создаем список из пользователей, у которых 100 и более оценок
film_fans_user_ids = ratings_filtered['userId'].tolist()

# Фильтруем наш исходный ДФ, используя список наших пользователей
film_fans_user_over100 = ratings[ ratings['userId'].isin(film_fans_user_ids) ]

# Считаем Lifetime
ratings_grouped1 = film_fans_user_over100.groupby('userId').agg([min, max])
ratings_grouped1['diff'] = ratings_grouped1['timestamp']['max'] - ratings_grouped1['timestamp']['min']
ratings_grouped1['diff'].mean() / 24 / 3600

455.2285713719898

### Задание №2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

**Необходимо сформировать две таблицы:**
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента  

_Обратите внимание, что в процессе объединения таблиц данные не должны теряться._

In [8]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [9]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [10]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [25]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [ ]:
# Вариант решение №1

In [33]:
full_table = (client_base
              .merge(rzd, on='client_id', how='left' )
              .merge(auto, on='client_id', how='left' )
              .merge(air, on='client_id', how='left' )
             )

small_table = full_table.drop('address', axis = 1, inplace = False)

small_table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [34]:
full_table

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [37]:
# Вариант решения №2
# Если считается, что по п.1 мы, например, не имели прав дооступа к таблице client_base, то:

first_table = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')
first_table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [39]:
second_table = client_base.merge(first_table, on='client_id', how='left')
second_table

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


### Задание №3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?


***

Во-первых, надо понимать, в какой момент считывались эти координаты, понимать, когда они требовались приложению и какое отношение они имеют к покупкам и визитам.  
Предположим, что визит - это **запуск приложения**. По количеству записей: 
* больше всего значений визитов
* затем количество пользователей
* затем количество покупок

Отсюда следует, что основная таблица - это визиты+покупки. Оптимальным, на-мой взгляд, был бы идентификатор по времени. Если таблица с координатами и таблица визитов+покупок имеет значение времени получения этих данных (данные о визите и координатах собирались в момент запуска приложения), то к "визиту" можно добавлять "координаты" по user_id + время получения данных.